In [4]:
import numpy as np
import copy


def process_data(path, unk_token="#UNK#"):
    tokens = []
    labels = []
    
    with open(path, encoding='utf-8') as f:
        raw = f.read()
        sentences = raw.strip().split('\n\n')

    for sentence in sentences:
        pairs = sentence.split('\n')
        inner_tokens = []
        inner_labels = []
        for pair in pairs:
            try:
                token, label = pair.split(' ')
            except:
                pass
            inner_tokens.append(token)
            inner_labels.append(label)

        tokens.append(inner_tokens)
        labels.append(inner_labels)

    unique_tokens = get_unique(tokens)
    unique_tokens = unique_tokens + [unk_token]
    unique_labels = get_unique(labels)
    
    return tokens, labels, unique_tokens, unique_labels


def dev_open(path):
  out = [[]]
  f = open(path, "r", encoding="utf-8")
  lines_in = f.readlines()
  for word in lines_in:
    if word == "\n":
      out.append([])
    else:
      out[-1].append(word.rstrip())
  return out[:-1]

def get_unique(nested_list):
    flattened_list = [item for sublist in nested_list for item in sublist]
    return sorted(list(set(flattened_list)))

def estimate_emission_matrix(unique_labels, unique_tokens, tokens, labels, unk_token="#UNK#", k=1):
    e_table = np.zeros((len(unique_labels), len(unique_tokens)+1))
    
    tag_counts = np.zeros(len(unique_labels))
    for label_seq in labels:
        for label in label_seq:
            tag_counts[unique_labels.index(label)] += 1
    
    for token_seq, label_seq in zip(tokens, labels):
        for token, label in zip(token_seq, label_seq):
            if token in unique_tokens:
                token_index = unique_tokens.index(token)
            else:
                token_index = unique_tokens.index(unk_token)  
            e_table[unique_labels.index(label)][token_index] += 1

    for i in range(len(unique_labels)):
        e_table[i, -1] += k / (tag_counts[i] + 1)
    
    e_table /= e_table.sum(axis=1)[:, np.newaxis]
    return e_table

def predict_labels_words(unique_labels, unique_tokens, e_table, test_data, unk_token="#UNK#"):
    predict_label = []
    
    for sentence in test_data:
        inner_predict = []
        for word in sentence:
            if word not in unique_tokens:
                word = unk_token
                pred_label = e_table[:, -1]
            else:
                pred_label = e_table[:, unique_tokens.index(word)]
            most_likely_label = unique_labels[np.argmax(pred_label)]
            inner_predict.append(most_likely_label)
        predict_label.append(inner_predict)
    
    return predict_label

def estimate_transition_matrix(unique_labels, labels):
    q_table = np.zeros((len(unique_labels)+1, len(unique_labels)+1))

    rows = ['START'] + unique_labels.copy()
    cols = unique_labels.copy() + ['STOP']

    for labels_seq in labels:
        x = copy.deepcopy(labels_seq)
        x.insert(0, 'START')
        x.append('STOP')

        for i in range(len(x)-1):
            cur_label = x[i]
            next_label = x[i+1]
            q_table[rows.index(cur_label)][cols.index(next_label)] += 1

    q_table /= q_table.sum(axis=1)[:, np.newaxis]
    return q_table

def viterbi_algorithm(unique_labels, unique_tokens, sentence, e_table, q_table, unk_token):
    n = len(sentence)
    sentence = [None] + sentence
    m = len(unique_labels)
    pi = np.zeros((n+2, m))

    unk_index = unique_tokens.index(unk_token)
    unk_emission_probs = e_table[:, unk_index] 

    for j in range(n):
        if sentence[j+1] in unique_tokens:
            cur_word = sentence[j+1]
            cur_word_index = unique_tokens.index(cur_word)
        else:
            cur_word = unk_token
            cur_word_index = unk_index

        for cur_index in range(m):
            current_e = unk_emission_probs[cur_index] if cur_word == unk_token else e_table[cur_index, cur_word_index]
            if j == 0:
                current_q = q_table[0, cur_index]
                pi[j+1, cur_index] = 1 * current_e * current_q
            else:
                max_prob = 0
                for vIndex in range(m):
                    current_q = q_table[vIndex+1, cur_index]
                    cur_prob = pi[j, vIndex] * current_e * current_q

                    if cur_prob > max_prob:
                        max_prob = cur_prob
                pi[j+1, cur_index] = max_prob
    
    max_prob = 0
    for prev_index in range(0, m):
        current_q = q_table[prev_index+1, -1]
        cur_prob = pi[n, prev_index] * current_q
        if cur_prob > max_prob:
            max_prob = cur_prob
    pi[n+1, -1] = max_prob
    
    y_star = [np.argmax(unk_emission_probs)] * (n+1)
    max_prob = 0
    
    for cur_index in range(0, m):
        current_q = q_table[cur_index+1, -1]
        cur_prob = pi[n, cur_index] * current_q
        
        if cur_prob > max_prob:
            max_prob = cur_prob
            y_star[n] = cur_index
    
    for j in range(n-1, 0, -1):
        max_prob = 0
        for cur_index in range(0, m):
            current_q = q_table[cur_index+1, y_star[j+1]]
            cur_prob = pi[j, cur_index] * current_q
            if cur_prob > max_prob:
                max_prob = cur_prob
                y_star[j] = cur_index
    
    labelled_preds = [unique_labels[y] for y in y_star[1:]]
    return labelled_preds

def predict_labels_sentences(unique_labels, unique_tokens, q_table, e_table, test_data, unk_token):
    total_preds = []

    for sentence in test_data:
        preds = viterbi_algorithm(unique_labels, unique_tokens, sentence, e_table, q_table, unk_token)
        total_preds.append(preds)

    return total_preds

def p1(test_data, predict_label_p1, output_path):
    with open(output_path, 'w', encoding='utf-8') as outp:
        for sentence, predictions in zip(test_data, predict_label_p1):
            for word, pos in zip(sentence, predictions):
                result = word + " " + pos + "\n"
                outp.write(result)
            outp.write('\n')


def p2(test_data, predict_label_p2, output_path):
    with open(output_path, 'w', encoding='utf-8') as outp:
        for sentence, predictions in zip(test_data, predict_label_p2):
            for word, pos in zip(sentence, predictions):
                result = word + " " + pos + "\n"
                outp.write(result)
            outp.write('\n')

In [7]:
def kthbest_viterbi(e_table, q_table, unique_tokens, unique_labels, unk_token, sentence, num):
    k = num
    n = len(sentence)
    sentence = [None] + sentence
    m = len(unique_labels)
    pi = np.zeros((n+2, m, k))

    for j in range(n):
        if sentence[j+1] in unique_tokens:
            cur_word = sentence[j+1]
        else:
            cur_word = unk_token

        for cur_index in range(0, m):
            current_e = e_table[cur_index, unique_tokens.index(cur_word)]
            if j == 0:
                current_q = q_table[0, cur_index]
                pi[j+1, cur_index, :] = 1 * current_e * current_q
            else:
                max_probs = []
                for prev_index in range(0, m):
                    for r in range(k):
                        current_q = q_table[prev_index+1, cur_index]
                        cur_prob = pi[j, prev_index, r] * current_e * current_q
                        max_probs.append(cur_prob)
                max_probs.sort(reverse=True)

                if len(max_probs) > k:
                    max_probs = max_probs[:k]
                pi[j+1, cur_index] = max_probs

    max_probs = []
    for prev_index in range(0, m):
        for r in range(k):
            current_q = q_table[prev_index+1, -1]
            cur_prob = pi[-1, prev_index, r] * current_q
            max_probs.append(cur_prob)

    max_probs.sort(reverse=True)
    if len(max_probs) > k:
        max_probs = max_probs[:k]
    pi[n+1, -1] = max_probs

    yxs = np.zeros((n+1, k), dtype=int) + unique_labels.index("O")
    max_probs = []

    def take_last(elem):
        return elem[-1]

    for prev_index in range(0, m):
        for r in range(k):
            current_q = q_table[prev_index+1, -1]
            cur_prob = pi[-1, prev_index, r] * current_q
            max_probs.append([prev_index, cur_prob])
    max_probs.sort(reverse=True, key=take_last)

    def removeRepeated(lst):
        new = []
        for elem in lst:
            if elem[1] != 0 and elem not in new:
                new.append(elem)
        return new

    max_probs = removeRepeated(max_probs)

    if len(max_probs) > k:
        max_probs = max_probs[:k]

    parents = [i[0] for i in max_probs]
    yxs[n, :len(max_probs)] = parents

    for j in range(n-1, 0, -1):
        max_probs = []
        for yx in yxs[j+1]:
            for cur_index in range(0, m):
                for r in range(k):
                    current_q = q_table[cur_index+1, yx]
                    cur_prob = pi[j, cur_index, r] * current_q
                    max_probs.append([cur_index, cur_prob])

        max_probs.sort(reverse=True, key=take_last)
        max_probs = removeRepeated(max_probs)

        if len(max_probs) > k:
            max_probs = max_probs[:k]

        parents = [i[0] for i in max_probs]
        yxs[j, :len(max_probs)] = parents

    labelled_preds = [unique_labels[y] for y in yxs.T[-1][1:]]
    return labelled_preds

def p3(input_path, output_path, unique_labels, unique_tokens, e_table, q_table, unk_token, num):
    total_preds = []
    data = dev_open(input_path)

    for sentence in data:
        preds = kthbest_viterbi(e_table, q_table, unique_tokens, unique_labels, unk_token, sentence, num)
        total_preds.append(preds)

    with open(output_path, 'w', encoding='utf-8') as outp:
        for _, (token, label) in enumerate(zip(data, total_preds)):
            for _, (word, pos) in enumerate(zip(token, label)):
                result = word + " " + pos + "\n"
                outp.write(result)
            outp.write('\n')
            

In [8]:

train_path = "RU/train"
unk_token = "#UNK#"
tokens, labels, unique_tokens, unique_labels = process_data(train_path, unk_token)
unique_labels = ['O', 'B-positive', 'B-neutral', 'B-negative', 'I-positive', 'I-neutral', 'I-negative']
unique_tokens.append(unk_token)


e_table = estimate_emission_matrix(unique_labels, unique_tokens, tokens, labels, unk_token, k=1)
q_table = estimate_transition_matrix(unique_labels, labels)

input_path = "RU/dev.in"
test_data = dev_open(input_path)

predict_label_p1 = predict_labels_words(unique_labels, unique_tokens, e_table, test_data, unk_token)
predict_label_p2 = predict_labels_sentences(unique_labels, unique_tokens, q_table, e_table, test_data, unk_token)

predict_p1_output_path = "RU/dev.p7.out"
predict_p2_output_path = "RU/dev.p8.out"
predict_p3_output_path = "RU/dev.p9.out"

p1(test_data, predict_label_p1, predict_p1_output_path)
p2(test_data, predict_label_p2, predict_p2_output_path)

num = 2
p3(input_path,  predict_p3_output_path, unique_labels, unique_tokens, e_table, q_table, unk_token, num)
